In [1]:
import time
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import log_loss
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn import datasets, metrics, model_selection, svm


# from notebooks/ directory
import sys, os

# ADDITIONS:
from src.RAI import DataBiasChecker, ModelBiasRanker, ModelBiasHandler


In [311]:
%%time
model_input_path = './data/raw/usa_00004.csv.gz'
ipums_select = pd.read_csv(model_input_path, compression='gzip', header=0, sep=',', quotechar='"', error_bad_lines=False,
                       usecols = ['SEX','EDUC','MARST','AGE',
                                  'YEAR','MET2013',
                                 'RACE','INCTOT','HISPAN'])


CPU times: user 3min 17s, sys: 48.1 s, total: 4min 5s
Wall time: 4min 14s


In [312]:
ipums_select.YEAR.max(), ipums_select.YEAR.min()

(2020, 1850)

In [305]:
ipums_select = ipums_select[ipums_select.YEAR == 2019]
ipums_select = ipums_select.drop(['YEAR'], axis = 1)

In [306]:
ipums_select['MARST'] = (ipums_select['MARST'].isin([1,2])).astype('int')
filt = ipums_select['MARST'] == 1
ipums_select.loc[filt, 'MARST'] = 'married'
ipums_select.loc[~filt, 'MARST'] = 'not_married'

In [307]:
ipums_select['EDUC'] = (ipums_select['EDUC'].isin([10,11])).astype('int')
filt = ipums_select['EDUC'] == 1
ipums_select.loc[filt, 'EDUC'] = 'college'
ipums_select.loc[~filt, 'EDUC'] = 'no_college'

In [308]:
ipums_select['SEX'] = (ipums_select['SEX'].isin([1])).astype('int')
filt = ipums_select['SEX'] == 1
ipums_select.loc[filt, 'SEX'] = 'Male'
ipums_select.loc[~filt, 'SEX'] = 'Female'

In [309]:
ipums_select

,MET2013,SEX,AGE,MARST,RACE,HISPAN,EDUC,INCTOT
12708071,0,Male,39,not_married,2,0,no_college,9000
12708072,0,Female,21,not_married,1,0,no_college,150
12708073,11500,Male,19,not_married,2,0,no_college,1400
12708074,11500,Male,77,not_married,1,0,no_college,22700
12708075,33660,Male,41,not_married,2,0,no_college,0
...,...,...,...,...,...,...,...,...
15947619,0,Female,63,not_married,1,0,no_college,117000
15947620,0,Male,45,not_married,1,0,no_college,14000
15947621,0,Female,85,not_married,1,0,no_college,18500
15947622,0,Female,67,married,1,0,college,49900


In [310]:
ipums_select.to_csv('./data/raw/ipums_processed_2019.csv', index=False)